# LSTM Forecasting for NGN/Euro Exchange Rate
To achieve your goals of forecasting NGN/EURO prices using LSTM and creating a user interface for future date predictions, I'll outline a comprehensive solution:

Solution Approach
1. Data Preparation
First, we need to properly prepare the time series data for LSTM modeling:

In [64]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime


In [65]:
df3= pd.read_csv("ngn_eur_data.csv")

In [66]:
df = df3.copy()
df = df.dropna()

In [71]:
df.head()

,Unnamed: 0,EUR_NGN_1. open,EUR_NGN_2. high,EUR_NGN_3. low,EUR_NGN_4. close,TRENDS_Euro to Naira
16,2014-12-01,222.37,230.14000,222.07001,229.63000,15.0
17,2014-12-02,229.62,229.67000,222.36000,222.48000,15.0
18,2014-12-03,222.48,222.59000,220.27000,220.37000,15.0
19,2014-12-04,220.36,224.03999,220.28000,221.64000,15.0
20,2014-12-05,221.63,225.00000,220.28000,220.28999,15.0


In [74]:

#  Rename columns
df.rename(columns={
    'Unnamed: 0': 'Date',
    'EUR_NGN_4. close': 'NGN_EURO',
    'TRENDS_Euro to Naira': 'Google_Trends'
}, inplace=True)


print(df.head())

          Date   NGN_EURO  Google_Trends
16  2014-12-01  229.63000           15.0
17  2014-12-02  222.48000           15.0
18  2014-12-03  220.37000           15.0
19  2014-12-04  221.64000           15.0
20  2014-12-05  220.28999           15.0


C:\Users\kefa\AppData\Local\Temp\ipykernel_1532\1288464019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={


In [75]:
df = df[['Date', 'Google_Trends', 'NGN_EURO']]

In [43]:
print(df.head())

          Date  Google_Trends   NGN_EURO
16  2014-12-01           15.0  229.63000
17  2014-12-02           15.0  222.48000
18  2014-12-03           15.0  220.37000
19  2014-12-04           15.0  221.64000
20  2014-12-05           15.0  220.28999


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2445 entries, 16 to 2460
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           2445 non-null   object 
 1   Google_Trends  2445 non-null   float64
 2   NGN_EURO       2445 non-null   float64
dtypes: float64(2), object(1)
memory usage: 76.4+ KB


In [77]:
df.shape

(2445, 3)

## Data Preparation

In [78]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Convert Date to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Check for missing values
print(df.isnull().sum())

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['NGN_EURO']])

Google_Trends    0
NGN_EURO         0
dtype: int64


## Create Time Series Sequences

In [79]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data)-seq_length-1):
        X.append(data[i:(i+seq_length), 0])
        y.append(data[i+seq_length, 0])
    return np.array(X), np.array(y)

# Define sequence length (e.g., 60 days)
seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

# Split into train/test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape for LSTM [samples, timesteps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

## Build and Train LSTM Model

In [80]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=1)

Epoch 1/50


C:\Users\kefa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 0.0027 - val_loss: 0.0040
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 1.6252e-04 - val_loss: 0.0045
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 1.5219e-04 - val_loss: 0.0057
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 1.2557e-04 - val_loss: 0.0058
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 1.1892e-04 - val_loss: 0.0073
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 1.3153e-04 - val_loss: 0.0062
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 9.8333e-05 - val_loss: 0.0073
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 9.7030e-05 - val_loss: 0.0105
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 9.0303e-05 - val_loss: 0.0077
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 8.2435e-05 - val_loss: 0.0086
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 8.3295e-05 - val_loss: 0.0071
Epoch 12/50
60/60 ━━━━━━━━━━━━━━━━

## Forecasting Function

In [81]:
def forecast_future_dates(model, last_sequence, future_dates, scaler):
    predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(len(future_dates)):
        # Predict next value
        next_pred = model.predict(current_sequence.reshape(1, seq_length, 1))
        
        # Store prediction
        predictions.append(next_pred[0,0])
        
        # Update sequence
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = next_pred
    
    # Inverse transform predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    return predictions

## Here's how to test it:

In [82]:
import numpy as np
from datetime import datetime, timedelta

# Assuming we have our trained model and scaler loaded
# model = load_model('ngn_euro_lstm.h5')
# scaler = joblib.load('scaler.pkl')

# Test the forecast_future_dates function
def test_forecast():
    # Get the last sequence from our data (most recent 60 days)
    last_sequence = scaled_data[-seq_length:]  # Using the scaled_data from earlier
    
    # Define our target date (correcting what appears to be a typo - 205 to 2025)
    target_date = datetime(2025, 4, 29).date()
    today = datetime.today().date()
    
    # Calculate days ahead (ensure it's a positive number)
    days_ahead = (target_date - today).days
    if days_ahead <= 0:
        print("Please select a date in the future")
        return
    
    # Generate all future dates between tomorrow and target date
    future_dates = [today + timedelta(days=i) for i in range(1, days_ahead+1)]
    
    # Get predictions
    predictions = forecast_future_dates(model, last_sequence, future_dates, scaler)
    
    # Get the specific prediction for our target date
    target_prediction = predictions[-1][0]
    
    print(f"Predicted NGN/EURO rate for {target_date}: {target_prediction:.4f}")
    
    return predictions

# Run the test
predictions = test_forecast()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
Predicted NGN/EURO rate for 2025-04-29: 1726.1964


### Saving the Model and Scaler

In [83]:
from keras.models import save_model
import joblib
import pickle

# Save the trained LSTM model
save_model(model, 'ngn_euro_lstm.h5')  # Saves in HDF5 format

# Save the scaler object
joblib.dump(scaler, 'ngn_euro_scaler.pkl') 

# Save the sequence length (important for reconstruction)
with open('seq_length.pkl', 'wb') as f:
    pickle.dump(seq_length, f)

### Saving the Last Sequence

In [84]:
# Save the last sequence of scaled data needed for future predictions
np.save('last_sequence.npy', scaled_data[-seq_length:])

### Complete Saving Function

In [86]:
def save_forecast_artifacts(model, scaler, scaled_data, seq_length):
    # Save model
    save_model(model, 'ngn_euro_lstm.h5')
    
    # Save scaler
    joblib.dump(scaler, 'ngn_euro_scaler.pkl')
    
    # Save sequence length
    with open('seq_length.pkl', 'wb') as f:
        pickle.dump(seq_length, f)
    
    # Save last sequence
    np.save('last_sequence.npy', scaled_data[-seq_length:])
    
    print("All artifacts saved successfully")

##  Create User Interface

### import streamlit as st
from datetime import datetime, timedelta

# Load trained model and scaler
model = load_model('ngn_euro_lstm.h5')
scaler = joblib.load('scaler.pkl')

st.title('NGN/EURO Exchange Rate Forecast')

# User input for future date
future_date = st.date_input("Select a future date for prediction:", 
                           min_value=datetime.today() + timedelta(days=1))

if st.button('Predict'):
    # Calculate days ahead
    days_ahead = (future_date - datetime.today().date()).days
    
    # Get last available sequence
    last_sequence = scaled_data[-seq_length:]
    
    # Generate predictions
    future_dates = [datetime.today() + timedelta(days=i) for i in range(1, days_ahead+1)]
    predictions = forecast_future_dates(model, last_sequence, future_dates, scaler)
    
    # Get the specific prediction for the selected date
    target_prediction = predictions[-1][0]
    
    st.success(f"Predicted NGN/EURO rate for {future_date}: {target_prediction:.4f}")